In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Load and normalize the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.2, random_state=42
)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

batch_size = 16
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class IrisModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(IrisModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim,bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def train_and_evaluate(hidden_dim, learning_rate, num_epochs):
    model = IrisModel(input_dim=4, hidden_dim=hidden_dim, output_dim=3)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        epoch_loss = 0
        for batch_features, batch_labels in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            
        avg_epoch_loss = epoch_loss / len(train_loader)
        
        # Evaluation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_features, batch_labels in test_loader:
                outputs = model(batch_features)
                _, predicted = torch.max(outputs, 1)
                total += batch_labels.size(0)
                correct += (predicted == batch_labels).sum().item()
            
        accuracy = 100 * correct / total
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, "
                  f"Loss: {avg_epoch_loss:.4f}, "
                  f"Test Accuracy: {accuracy:.2f}%")
    
    # Store weight matrix after training
    with torch.no_grad():
        fc1_weight = model.fc1.weight.data.clone()
    
    return model, fc1_weight


In [2]:
redundancy_settings = [2, 4, 6, 8, 10, 20, 40]
learning_rate = 0.5
num_epochs = 100

# Dictionary to store weight matrices for each redundancy setting
weight_matrices = {}

# Train the model for each redundancy setting
for hidden_dim in redundancy_settings:
    print(f"\nTraining with hidden_dim = {hidden_dim}")
    model, fc1_weight = train_and_evaluate(hidden_dim, learning_rate, num_epochs)
    
    # Store weight matrices
    weight_matrices[hidden_dim] = fc1_weight

# Print all stored weight matrices at the end
print("\nFinal Weight Matrices for all redundancy settings:")
for hidden_dim in redundancy_settings:
    print(f"\nHidden dimension: {hidden_dim}")
    print(f"FC1 weight matrix (dimension {hidden_dim}):")
    print(weight_matrices[hidden_dim])



Training with hidden_dim = 2
Epoch 10/100, Loss: 0.1503, Test Accuracy: 100.00%
Epoch 20/100, Loss: 0.0912, Test Accuracy: 100.00%
Epoch 30/100, Loss: 0.0886, Test Accuracy: 96.67%
Epoch 40/100, Loss: 0.0994, Test Accuracy: 93.33%
Epoch 50/100, Loss: 0.0853, Test Accuracy: 96.67%
Epoch 60/100, Loss: 0.0663, Test Accuracy: 100.00%
Epoch 70/100, Loss: 0.0746, Test Accuracy: 100.00%
Epoch 80/100, Loss: 0.0625, Test Accuracy: 96.67%
Epoch 90/100, Loss: 0.0642, Test Accuracy: 96.67%
Epoch 100/100, Loss: 0.0692, Test Accuracy: 96.67%

Training with hidden_dim = 4
Epoch 10/100, Loss: 0.1833, Test Accuracy: 100.00%
Epoch 20/100, Loss: 0.1181, Test Accuracy: 100.00%
Epoch 30/100, Loss: 0.1001, Test Accuracy: 100.00%
Epoch 40/100, Loss: 0.0830, Test Accuracy: 100.00%
Epoch 50/100, Loss: 0.0830, Test Accuracy: 100.00%
Epoch 60/100, Loss: 0.0744, Test Accuracy: 100.00%
Epoch 70/100, Loss: 0.0766, Test Accuracy: 100.00%
Epoch 80/100, Loss: 0.0986, Test Accuracy: 100.00%
Epoch 90/100, Loss: 0.0624,